# Motivation

The motivation for creating prediction and ranking tasks for EIE datasets was to extend the dataset ecosystem available for algorithmic fairness research in two directions: the education domain and the dataset from Eastern Europe (Ukraine). We obtained the data from publicly available Open Data resource https://zno.testportal.com.ua/opendata.


The tasks were created by Dr. Julia Stoyanovich and Andrew Bell from the Center for Responsible AI, New York University, and Tetiana Zakharchenko, Nazarii Drushchak, Oleksandra Konopatska, and Olha Liuba from Ukrainian Catholic University.


The creation of the dataset was funded by the Center for Responsible AI, New York University.

# Composition of Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

We will load and work with datasets from 2016 to 2021 since they have a comparable structure. We downloadedoriginal datasets from Open Data Resource and uploaded to HuggingFace for easiness of usage.

## Loading and merging

Load datasets from hugging face

In [2]:
# load datasets
datasets = []
years = range(2016, 2022)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='utf-8')
    except:
        dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='Windows 1251')
    datasets.append(dataset)

EIE 2016 Loading...
EIE 2017 Loading...
EIE 2018 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_72720/2552149954.py:11: DtypeWarning: Columns (119) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='utf-8')


EIE 2019 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_72720/2552149954.py:13: DtypeWarning: Columns (100,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


EIE 2020 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_72720/2552149954.py:13: DtypeWarning: Columns (96,97,98,100,102,103,104,105,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


EIE 2021 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_72720/2552149954.py:11: DtypeWarning: Columns (117,118,119,121,123,124,125,126,137,138,139,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='utf-8')


Names of all features are the same across all years, but can have some difference with capital and lower cases. That's why we lower all of them. Also, we add a new column – a corresponding year. For further research we add the column with age.

In [3]:
#lowercase all columns and add year as attribute
for year, dataset in zip(years, datasets):
    dataset.columns = [col.lower() for col in dataset.columns]
    dataset['year'] = year
    dataset['age'] = year - dataset['birth']

## Unifiying column names

### 2016, 2017 and 2018

compare 2016 (dataset[0]) and 2018 (dataset[2]) structure

In [4]:
set(datasets[0].columns)-set(datasets[2].columns)

{'frball100',
 'frptareaname',
 'frptname',
 'frptregname',
 'frpttername',
 'frtest',
 'frteststatus',
 'rusball100',
 'rusptareaname',
 'rusptname',
 'rusptregname',
 'ruspttername',
 'rustest',
 'rusteststatus',
 'spball100',
 'spptareaname',
 'spptname',
 'spptregname',
 'sppttername',
 'sptest',
 'spteststatus'}

In [5]:
set(datasets[2].columns)-set(datasets[0].columns)

{'bioball',
 'bioball12',
 'chemball',
 'chemball12',
 'classlangname',
 'classprofilename',
 'deuball',
 'deuball12',
 'deudpalevel',
 'engball',
 'engball12',
 'engdpalevel',
 'fraball',
 'fraball100',
 'fraball12',
 'fradpalevel',
 'fraptareaname',
 'fraptname',
 'fraptregname',
 'frapttername',
 'fratest',
 'frateststatus',
 'geoball',
 'geoball12',
 'histball',
 'mathball',
 'physball',
 'physball12',
 'spaball',
 'spaball100',
 'spaball12',
 'spadpalevel',
 'spaptareaname',
 'spaptname',
 'spaptregname',
 'spapttername',
 'spatest',
 'spateststatus',
 'tertypename',
 'ukrball'}

In [6]:
datasets[0].columns = [col[:2]+'a'+col[2:] if col.startswith('fr') or col.startswith('sp') else col for col in datasets[0].columns]

compare 2017 and 2018

In [7]:
datasets[1]

,outid,stid,birth,sextypename,regname,areaname,tername,regtypename,tertypename,classprofilename,...,rustest,rusteststatus,rusball100,rusball12,rusptname,rusptregname,rusptareaname,ruspttername,year,age
0,EDDDC0A3-C615-4101-96B3-9F1A4C654A2C,14809269-9F28-454C-8B3B-01DE067C91F3,1999,жіноча,Житомирська область,Новоград-Волинський район,смт Городниця,Випускник минулих років,місто,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,18
1,3528FBEB-5ABE-4956-B715-68FABAE0A566,C5B85614-D95B-4029-B3E4-920246DA7B17,1999,чоловіча,Сумська область,Сумська область,м.Лебедин,"Учень (слухач, студент) професійно-технічного,...",місто,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,18
2,1BA2DD60-B125-4DB0-875A-9AB8E54B4411,E1BB0166-3FFE-4BA0-BFB7-D60EC8E8CD8E,1991,чоловіча,Миколаївська область,м.Миколаїв,Корабельний район міста,Випускник минулих років,місто,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,26
3,29BD1575-16B2-4EAD-B2AB-DC2A3365DE1E,574350E1-DDA9-47A8-ADAA-6F8915BD2A05,2000,чоловіча,Донецька область,Донецька область,м.Покровськ,Випускник загальноосвітнього навчального закла...,місто,Фізико-математичний,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,17
4,FF8BFD9C-837B-4F38-A321-C9AC264F83E6,4E01CF4C-3596-4178-A44D-556197D73E7B,2000,чоловіча,Івано-Франківська область,Івано-Франківська область,м.Івано-Франківськ,"Учень (слухач, студент) професійно-технічного,...",місто,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240884,D7D5DDD6-473F-4047-B673-88AC5922BCFB,EA2E4E4D-9672-48BA-A4BF-E77DD3A83EC4,1999,жіноча,м.Київ,м.Київ,Шевченківський район міста,Випускник загальноосвітнього навчального закла...,місто,Інформаційно-технологічний,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,18
240885,678C3946-388B-4921-8679-C0788F21AAD4,97B6798B-B6BB-42B9-B971-91CD92053041,1999,жіноча,Харківська область,м.Харків,Московський район міста,Випускник загальноосвітнього навчального закла...,місто,Інший,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,18
240886,9C4F308A-1A47-46A4-A192-CAFC3DF0A726,A84568A5-DFA5-45D2-ADD3-5196E1A7A72E,2000,чоловіча,Херсонська область,Великоолександрівський район,с.Чарівне,Випускник загальноосвітнього навчального закла...,село,Універсальний,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,17
240887,DECAE4A3-0782-4565-9A00-95393AA7FCD9,A1113632-D542-43B8-BB44-E56FE6446951,1998,жіноча,Рівненська область,Гощанський район,с.Бабин,Випускник минулих років,село,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,19


In [8]:
set(datasets[1].columns)-set(datasets[2].columns)

{'rusball100',
 'rusball12',
 'rusptareaname',
 'rusptname',
 'rusptregname',
 'ruspttername',
 'rustest',
 'rusteststatus',
 'stid'}

In [9]:
set(datasets[2].columns)-set(datasets[1].columns)

{'bioball',
 'chemball',
 'deuball',
 'deudpalevel',
 'engball',
 'engdpalevel',
 'fraball',
 'fradpalevel',
 'geoball',
 'histball',
 'mathball',
 'physball',
 'spaball',
 'spadpalevel',
 'ukrball'}

In [10]:
datasets[1] = datasets[1].drop(columns=['stid'])

Finally, we can concatenate all datasets.

In [11]:
# concat datasets
df_all = pd.concat(datasets)

Save datasets to csv format.

In [12]:
df_all.to_csv('zno_all.csv',index=False)

Having created a dataset once, we can load it immediately in the future

In [43]:
df_all = pd.read_csv('zno_all.csv')

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_72720/2055058606.py:1: DtypeWarning: Columns (15,24,33,42,50,58,66,73,78,79,80,81,82,83,84,87,92,93,94,95,96,97,98,99,100,101,102,103,104,105,108,109,110,127,129,131,133,136,137,138,142,143,144,145,146,147,148,149,150,153,154,155,156) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv('zno_all.csv')


## Exploring dataset

In [44]:
df_all.head()

,outid,birth,sextypename,regname,areaname,tername,regtypename,eoname,eotypename,eoregname,...,mathdpalevel,mathsttest,mathstlang,mathstteststatus,mathstball12,mathstball,mathstptname,mathstptregname,mathstptareaname,mathstpttername
0,a99c6c63-aa70-4aec-ba42-370f7261e857,1998,чоловіча,Запорізька область,Мелітопольський район,с.Терпіння,Випускник загальноосвітнього навчального закла...,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",колегіум,Запорізька область,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c3136421-569e-422e-ae8f-41c4c931fd70,1998,жіноча,Хмельницька область,Красилівський район,м.Красилів,Випускник загальноосвітнього навчального закла...,Красилівська загальноосвітня школа I-III ступе...,середня загальноосвітня школа,Хмельницька область,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fbd0e280-4b77-4f34-942d-00bc850c2992,1995,жіноча,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,Випускник минулих років,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30de395e-7a74-452a-8370-6856d240fbfb,1999,чоловіча,Чернівецька область,м.Чернівці,Шевченківський район міста,Випускник загальноосвітнього навчального закла...,Чернівецька спеціалізована школа І-ІІІ ступені...,спеціалізована школа,Чернівецька область,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52a40c74-374e-4744-bd6f-938ab7a4a641,1997,чоловіча,Миколаївська область,Врадіївський район,с.Кумарі,"Учень (слухач, студент) професійно-технічного,...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967014 entries, 0 to 1967013
Columns: 157 entries, outid to mathstpttername
dtypes: float64(29), int64(3), object(125)
memory usage: 2.3+ GB


In [46]:
dict(df_all.dtypes)

{'outid': dtype('O'),
 'birth': dtype('int64'),
 'sextypename': dtype('O'),
 'regname': dtype('O'),
 'areaname': dtype('O'),
 'tername': dtype('O'),
 'regtypename': dtype('O'),
 'eoname': dtype('O'),
 'eotypename': dtype('O'),
 'eoregname': dtype('O'),
 'eoareaname': dtype('O'),
 'eotername': dtype('O'),
 'eoparent': dtype('O'),
 'ukrtest': dtype('O'),
 'ukrteststatus': dtype('O'),
 'ukrball100': dtype('O'),
 'ukrball12': dtype('float64'),
 'ukrptname': dtype('O'),
 'ukrptregname': dtype('O'),
 'ukrptareaname': dtype('O'),
 'ukrpttername': dtype('O'),
 'histtest': dtype('O'),
 'histlang': dtype('O'),
 'histteststatus': dtype('O'),
 'histball100': dtype('O'),
 'histball12': dtype('float64'),
 'histptname': dtype('O'),
 'histptregname': dtype('O'),
 'histptareaname': dtype('O'),
 'histpttername': dtype('O'),
 'mathtest': dtype('O'),
 'mathlang': dtype('O'),
 'mathteststatus': dtype('O'),
 'mathball100': dtype('O'),
 'mathball12': dtype('float64'),
 'mathptname': dtype('O'),
 'mathptregna

In [47]:
df_all.dropna(how = 'all', inplace = True)

In [48]:
df_all

,outid,birth,sextypename,regname,areaname,tername,regtypename,eoname,eotypename,eoregname,...,mathdpalevel,mathsttest,mathstlang,mathstteststatus,mathstball12,mathstball,mathstptname,mathstptregname,mathstptareaname,mathstpttername
0,a99c6c63-aa70-4aec-ba42-370f7261e857,1998,чоловіча,Запорізька область,Мелітопольський район,с.Терпіння,Випускник загальноосвітнього навчального закла...,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",колегіум,Запорізька область,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c3136421-569e-422e-ae8f-41c4c931fd70,1998,жіноча,Хмельницька область,Красилівський район,м.Красилів,Випускник загальноосвітнього навчального закла...,Красилівська загальноосвітня школа I-III ступе...,середня загальноосвітня школа,Хмельницька область,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fbd0e280-4b77-4f34-942d-00bc850c2992,1995,жіноча,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,Випускник минулих років,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30de395e-7a74-452a-8370-6856d240fbfb,1999,чоловіча,Чернівецька область,м.Чернівці,Шевченківський район міста,Випускник загальноосвітнього навчального закла...,Чернівецька спеціалізована школа І-ІІІ ступені...,спеціалізована школа,Чернівецька область,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52a40c74-374e-4744-bd6f-938ab7a4a641,1997,чоловіча,Миколаївська область,Врадіївський район,с.Кумарі,"Учень (слухач, студент) професійно-технічного,...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1967009,e62710ee-74fa-4085-b066-f38fc37ceae7,2003,жіноча,Рівненська область,Демидівський район,смт Демидівка,Учень (слухач) закладу професійної (професійно...,Вище професійне училище №25 смт.Демидівка,заклад професійної (професійно-технічної) освіти,Рівненська область,...,стандарт(академічний),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1967010,b2e4de44-7ebd-4163-8952-f988a9de1f67,1989,жіноча,Харківська область,Кегичівський район,смт Кегичівка,Учень (слухач) закладу професійної (професійно...,Професійно-технічне училище №60 смт Кегичівка ...,заклад професійної (професійно-технічної) освіти,Харківська область,...,NaN,Математика (завдання рівня стандарту),українська,Не з’явився,NaN,NaN,Красноградський ліцей №4 Красноградської міськ...,Харківська область,Красноградський район,м.Красноград
1967011,e007c190-1fe1-4658-9eb0-f7cf489fc015,2004,жіноча,Одеська область,Одеська область,м.Білгород-Дністровський,Студент закладу вищої освіти,"ОКУ ""Білгород-Дністровський медичний фаховий к...",заклад фахової передвищої освіти,Одеська область,...,NaN,Математика (завдання рівня стандарту),українська,Не з’явився,NaN,NaN,Загальноосвітня школа І-III ступенів № 1 м. Бі...,Одеська область,м.Білгород-Дністровський,м.Білгород-Дністровський
1967012,2668b9f3-ddf9-4062-acd7-fdf6e2ca8cc8,2004,чоловіча,Вінницька область,Гайсинський район,с.Чечелівка,Випускник загальноосвітнього навчального закла...,Середня загальноосвіня школа I-III ступенів с....,середня загальноосвітня школа,Вінницька область,...,профільний,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
for col in df_all.columns:
    if '100' in col:
        df_all[col] = pd.to_numeric(df_all[col].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))

In [67]:
df_all['ukrteststatus'].unique()

array(['Зараховано', "Не з'явився", 'Не подолав поріг', nan, 'Анульовано',
       'Не обрано 100-200'], dtype=object)

In [66]:
for col in df_all.columns:
    if 'teststatus' in col:
        df_all[col] = df_all[col].str.replace('’', "'")
        df_all[col] = df_all[col].replace('Не зареєстровано для проходження основної сесії', np.nan)
        df_all[col] = df_all[col].replace('Не склав', 'Не подолав поріг')
        df_all[col] = df_all[col].replace('Отримав результат', 'Зараховано')

In [69]:
dict(df_all.dtypes)

{'outid': dtype('O'),
 'birth': dtype('int64'),
 'sextypename': dtype('O'),
 'regname': dtype('O'),
 'areaname': dtype('O'),
 'tername': dtype('O'),
 'regtypename': dtype('O'),
 'eoname': dtype('O'),
 'eotypename': dtype('O'),
 'eoregname': dtype('O'),
 'eoareaname': dtype('O'),
 'eotername': dtype('O'),
 'eoparent': dtype('O'),
 'ukrtest': dtype('O'),
 'ukrteststatus': dtype('O'),
 'ukrball100': dtype('float64'),
 'ukrball12': dtype('float64'),
 'ukrptname': dtype('O'),
 'ukrptregname': dtype('O'),
 'ukrptareaname': dtype('O'),
 'ukrpttername': dtype('O'),
 'histtest': dtype('O'),
 'histlang': dtype('O'),
 'histteststatus': dtype('O'),
 'histball100': dtype('float64'),
 'histball12': dtype('float64'),
 'histptname': dtype('O'),
 'histptregname': dtype('O'),
 'histptareaname': dtype('O'),
 'histpttername': dtype('O'),
 'mathtest': dtype('O'),
 'mathlang': dtype('O'),
 'mathteststatus': dtype('O'),
 'mathball100': dtype('float64'),
 'mathball12': dtype('float64'),
 'mathptname': dtype('

## Unifying 'tertypename' feature

Since in 2021 'tertypename' has 3 territory types: city (місто), town (селище міського типу) and village (селище, село), and before we had only 2: city (місто) and village (селище, село), we reconstruct the correct type for 2018-2020 period.

In [71]:
def encode_tertypename(x):
    if x['tername'].startswith('м.'):
        return 'місто'
    elif x['tername'].startswith('смт'):
        return 'селище міського типу'
    elif x['tername'].startswith('с.'):
        return 'селище, село'
    elif x['tername'].startswith('с-ще'):
         return 'селище, село'
    elif x['areaname'].startswith('м.'):
        return 'місто'
    return None
        
df_all['tertypename'] = df_all.apply(encode_tertypename,axis=1)

In [72]:
df_all.tertypename.value_counts()

tertypename
місто                   1381902
селище, село             424923
селище міського типу     160188
Name: count, dtype: int64

## Unifying 'regtypename' feature

In [73]:
regtypename_unique = df_all.regtypename.unique()
sorted(regtypename_unique)

['Випускник загальноосвітнього навчального закладу 2016 року',
 'Випускник загальноосвітнього навчального закладу 2017 року',
 'Випускник загальноосвітнього навчального закладу 2021 року',
 'Випускник закладу загальної середньої освіти 2018 року',
 'Випускник закладу загальної середньої освіти 2019 року',
 'Випускник минулих років',
 'Випускник, який здобуде в 2016 році повну загальну середню освіту в навчальному закладі іншої держави',
 'Випускник, який здобуде в 2017 році повну загальну середню освіту в навчальному закладі іншої держави',
 'Випускник, який здобуде в 2018 році повну загальну середню освіту в навчальному закладі іншої держави',
 'Випускник, який здобуде в 2019 році повну загальну середню освіту в закордонному закладі освіти',
 'Випускник, який здобуде в 2020 році повну загальну середню освіту в закордонному закладі освіти',
 'Випускник, який здобуде в 2021 році повну загальну середню освіту в навчальному закладі іншої держави',
 'Студент закладу вищої освіти',
 'Устано

In [74]:
regtypename_dict = {
    'Випускник української школи поточного року': ['Випускник загальноосвітнього навчального закладу 2016 року','Випускник загальноосвітнього навчального закладу 2017 року',
                         'Випускник загальноосвітнього навчального закладу 2021 року', 'Випускник закладу загальної середньої освіти 2018 року',
                         'Випускник закладу загальної середньої освіти 2019 року', 'випускник закладу загальної середньої освіти 2020 року'],
    'Випускник іноземної школи': ['Випускник, який здобуде в 2016 році повну загальну середню освіту в навчальному закладі іншої держави', 'Випускник, який здобуде в 2017 році повну загальну середню освіту в навчальному закладі іншої держави',
                                  'Випускник, який здобуде в 2018 році повну загальну середню освіту в навчальному закладі іншої держави', 'Випускник, який здобуде в 2019 році повну загальну середню освіту в закордонному закладі освіти',
                                  'Випускник, який здобуде в 2020 році повну загальну середню освіту в закордонному закладі освіти', 'Випускник, який здобуде в 2021 році повну загальну середню освіту в навчальному закладі іншої держави'],
    'Учень коледжу закладу професійної (професійно-технічної) або вищої освіти': ['Учень (слухач) закладу професійної (професійно-технічної) освіти', 'Учень (слухач, студент) професійно-технічного, вищого навчального закладу', 'Студент закладу вищої освіти'],
    'Випускник минулих років': ['Випускник минулих років'],
    'Установа виконання покарань':['Установа виконання покарань']
}

In [75]:
def encode_regtypename(x):
    for type, possible_types in regtypename_dict.items():
        if x in possible_types:
            return type
    return None

df_all.regtypename = df_all.regtypename.map(encode_regtypename)

In [76]:
df_all.regtypename.unique()

array(['Випускник української школи поточного року',
       'Випускник минулих років',
       'Учень коледжу закладу професійної (професійно-технічної) або вищої освіти',
       'Установа виконання покарань', 'Випускник іноземної школи'],
      dtype=object)

## Drop Math Standard test and UkrsubTest

It was new experiment in 2021, that failed since due to COVID, it was cancelled.

In [77]:
df_all = df_all.drop(columns=[col for col in df_all.columns if col.startswith('mathst')])

In [78]:
df_all = df_all.drop(columns=['ukrsubtest'])

## Change UML in 2021

In 2021 Ukrtest was renamed to UMLtest, and was created the new test only with Ukrainian language

In [79]:
def prep_ukr_uml(x, name):
    if x['year']<2021:
        return x[name]
    return x[name.replace('ukr','uml')]

for name in [col for col in df_all.columns if col.startswith('ukr')]:
    df_all[name] = df_all.apply(prep_ukr_uml,axis=1,args=[name])

In [80]:
df_all = df_all.drop(columns=[col for col in df_all.columns if col.startswith('uml')])

## Drop Ball12

They are ranking scores for different scales.

In [81]:
df_all = df_all.drop(columns=[col for col in df_all.columns if '12' in col])

In [82]:
df_all = df_all.drop(columns = 'birth')

In [83]:
df_all = df_all.drop(columns=[col for col in df_all.columns if 'dpalevel' in col])

In [84]:
list(df_all.columns)

['outid',
 'sextypename',
 'regname',
 'areaname',
 'tername',
 'regtypename',
 'eoname',
 'eotypename',
 'eoregname',
 'eoareaname',
 'eotername',
 'eoparent',
 'ukrtest',
 'ukrteststatus',
 'ukrball100',
 'ukrptname',
 'ukrptregname',
 'ukrptareaname',
 'ukrpttername',
 'histtest',
 'histlang',
 'histteststatus',
 'histball100',
 'histptname',
 'histptregname',
 'histptareaname',
 'histpttername',
 'mathtest',
 'mathlang',
 'mathteststatus',
 'mathball100',
 'mathptname',
 'mathptregname',
 'mathptareaname',
 'mathpttername',
 'phystest',
 'physlang',
 'physteststatus',
 'physball100',
 'physptname',
 'physptregname',
 'physptareaname',
 'physpttername',
 'chemtest',
 'chemlang',
 'chemteststatus',
 'chemball100',
 'chemptname',
 'chemptregname',
 'chemptareaname',
 'chempttername',
 'biotest',
 'biolang',
 'bioteststatus',
 'bioball100',
 'bioptname',
 'bioptregname',
 'bioptareaname',
 'biopttername',
 'geotest',
 'geolang',
 'geoteststatus',
 'geoball100',
 'geoptname',
 'geoptreg

In [85]:
df_all.to_csv('zno_preprocessed.csv', index=False)

In [86]:
df_all.shape

(1967014, 119)